In [106]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node ad6b0acc-2bf0-4a27-bb7a-64d9d8bda21d

In [107]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm
import time
import KalmanFilter

In [108]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [109]:
#Thymio control
def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def rotate(theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

async def stop_motor():
    await node.set_variables(motors(0,0))

async def get_proximity_values():
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

# Run the asynchronous function and print the result
proximity_values = await get_proximity_values()
print(f"Front Proximity Sensor Value: {proximity_values}")

Front Proximity Sensor Value: [0, 0, 0, 0, 0]


In [110]:
## Parameters for local navigation
threshold_obst = 3500 
threshold_loc = 2500
local_motor_speed = 100
threshold_obst_list = [3200, 3600, 3600, 3600, 3200]

async def local_navigation():
    threshold_obst = 1000
    threshold_loc = 800
    local_motor_speed = 100
    threshold_obst_list = [3200, 3600, 3600, 3600, 3200]
    sens = await get_proximity_values()

    # Follow the obstacle by the left
    if (sens[0] + sens[1]) > (sens[4] + sens[3]):
        await bypass('right', sens, threshold_loc, local_motor_speed)

    # Follow the obstacle by the right    
    else:
        await bypass('left', sens, threshold_loc, local_motor_speed)

async def bypass(leftright, sens, threshold_loc, local_motor_speed):
    if leftright == "right":
        while sum(sens[i] > threshold_obst for i in range(0, 5)) > 0:
            print("Turn right")
            await rotate(np.pi/6/2, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()
            print(sens)

        while sens[0] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()
        
        

    elif leftright == "left":
        while sum(sens[i] > threshold_loc for i in range(0, 5)) > 0:
            print("Turn left")
            await rotate(-np.pi/6, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()

        while sens[4] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()

    if(leftright=="right" and sens[0] < threshold_loc):
        await forward(local_motor_speed)
        time.sleep(2)
        await rotate(-np.pi/4, 100)
        awa
    

    await forward(local_motor_speed)
    #time.sleep(2)
    await stop_motor()

In [113]:
thymio_data = []
#acquire_data = True
Ts = 0.1

def get_data():
    thymio_data.append({"sensor":node["prox.horizontal"],
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

async def get_speed():
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, get_data) # it auto-starts, no need of rt.start()

    try:
        await client.sleep(Ts)
    finally:
        rt.stop() # better in a try/finally block to make sure the program ends!
        node.send_set_variables(motors(0, 0))

def pos_estimation(left_speed, right_speed, dt, pos0_x, pos0_y, theta0):
    real_thymio_speed = 25.5 #mm/s
    real_thymio_angular_speed = 0.38 #rad/s
    #speed estimation
    # speed_measured = (right_speed[-1] + left_speed[-1])/2
    # speed = (speed_measured * 52.5) / real_thymio_speed 
    # angular_speed_measured = (right_speed[-1] - left_speed[-1])/2
    # angular_speed = (angular_speed_measured * 50) / real_thymio_angular_speed
    speed_measured = (right_speed + left_speed)/2
    #speed = (speed_measured * 50) / real_thymio_speed 
    speed = (speed_measured * real_thymio_speed) / 50
    angular_speed_measured = (right_speed - left_speed)/2
    #angular_speed = (angular_speed_measured * 50) / real_thymio_angular_speed
    angular_speed = (angular_speed_measured * real_thymio_angular_speed) / 50
    #print("left ", left_speed, "right ", right_speed, "angular speed measured", angular_speed_measured, "angular speed ", angular_speed, "speed ", speed, "temps ", dt)

    #position estimation
    theta = theta0 + angular_speed * dt
    pos_x = pos0_x + speed * np.cos(theta) * dt
    pos_y = pos0_y + speed * np.sin(theta) * dt

    return speed, pos_x, pos_y, theta #rajouter angular speed

In [114]:
#MAIN CODE : 

#parameters
path_step = 0 #This is the index of where we are in the path.
path_coord = np.array([[1,2], [2,3], [3,4]])
b = np.array([3.2, 4.5])
epsilon = 0.5
local_obstacle = False
threshold_obst = 3500 #threshold for obstacle detection
pos_x, pos_y, theta = 0, 0, 0
speed_history = []
thymio_data.clear()

#to implement in a different way thanks to filtering : 
robot_position = 1
end_position = 2

async def main():

    global local_obstacle, pos_x, pos_y, theta, dt
    start_time = time.time()

    while(robot_position != end_position):
        sens = await get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True

        if(local_obstacle):
            await local_navigation()
            start_time = time.time() #reset the timer          
        
        #if(find_index_path_if_close(path_coord,b,epsilon) and time.time-start_time>2): 
            #path_step=find_index_path_if_close(path_coord, position_robot, epsilon)
            #local_navigation=False

        if not local_obstacle: 
            node.send_set_variables(motors(50, 50))
            await get_speed()
            #if len(thymio_data) == 0: return pos0_x, pos0_y, theta0
    
            #current variables
            #left_speed = [x["left_speed"] for x in thymio_data]
            #right_speed = [x["right_speed"] for x in thymio_data]
            left_speed = node["motor.left.speed"]
            right_speed = node["motor.right.speed"]
            dt = time.time() - start_time 

            speed, pos_x, pos_y, theta = pos_estimation(left_speed, right_speed, dt, pos_x, pos_y, theta)
            start_time = time.time()
            speed_history.append(speed)
            print(round(pos_x/10,2), round(pos_y/10,2), round((theta*180)/np.pi,2))

await main()

std_speed = np.std(speed_history)
print("The speed variance in mm^2/s^2 is {}".format(std_speed))

left  0 right  0 angular speed measured 0.0 angular speed  0.0 speed  0.0 temps  0.5740997791290283
0.0 0.0 0.0
left  0 right  0 angular speed measured 0.0 angular speed  0.0 speed  0.0 temps  0.10572695732116699
0.0 0.0 0.0
left  34 right  34 angular speed measured 0.0 angular speed  0.0 speed  17.34 temps  0.10699200630187988
0.19 0.0 0.0
left  39 right  39 angular speed measured 0.0 angular speed  0.0 speed  19.89 temps  0.11734986305236816
0.42 0.0 0.0
left  45 right  41 angular speed measured -2.0 angular speed  -0.0152 speed  21.93 temps  0.10681390762329102
0.65 -0.0 -0.09
left  43 right  40 angular speed measured -1.5 angular speed  -0.0114 speed  21.165 temps  0.10617971420288086
0.88 -0.0 -0.16
left  40 right  39 angular speed measured -0.5 angular speed  -0.0038 speed  20.145 temps  0.10360288619995117
1.09 -0.0 -0.18
left  47 right  46 angular speed measured -0.5 angular speed  -0.0038 speed  23.715 temps  0.10798978805541992
1.34 -0.0 -0.21
left  55 right  49 angular speed

CancelledError: 

In [80]:
await node.set_variables(motors(0, 0))


In [ ]:
KalmanFilter.kalman_filter(pos_x, pos_y, theta, q_nu, r_nu, speed_history, dt)

In [ ]:
await tdmclient.notebook.stop()

ValueError: Function <function _pre_run_cell at 0x7f8be4798d30> is not registered as a pre_run_cell callback

In [55]:
await node.set_variables(motors(50, 55))
await client.sleep(10)
await node.set_variables(motors(0, 0))

In [61]:
await node.set_variables(motors(-50, 55))
await client.sleep(10)
await node.set_variables(motors(0, 0))